# Pie in The Sky: Food Syle Transfer with Cycle-GAN 

This file contains the ***training*** code of project of [ECE 285f](https://www.math.u-bordeaux.fr/~cdeledal/teaching.php#learningFA18) 2018.
We named this project as `Pie in The Sky` though it is not purely about food generation. But it can generate one type of food from the other, so it makes some `Pie in The Sky`. Actually, we truely get a food generator during the training of model. 


# Prerequisites
- Python 3.3 or above
- [Pytorch 0.4.0](torch.org)
- [Torchvision](https://github.com/pytorch/vision)
- [visdom visualizer 0.1.8.5](https://github.com/facebookresearch/visdom)
- [dominate 2.3.5](https://github.com/Knio/dominate)

For UCSD `DSMLP` user, you only need to install part of the package with the following command:
```bash
pip3 install visdom==0.1.8.5 --user
pip3 install dominate==2.3.5 --user
pip3 install torchvision --user
```

# Quick Start
In this project, we adopt [visdom](https://github.com/facebookresearch/visdom) for realtime visilization, which enable users to monitor the training process just in thier laptop, without cope another from the server.
So first, launch the visdom server. `Otherwise, the code will raise error!`
```bash
python3 -m visdom.server
```

Or, run it in background
```bash
nohup python3 -m visdom.server
```

## Important note
- Before testing, make sure that your ***visdom server*** has been successfully launched using the command above, otherwise you may encounter an ***error***. 

In [ ]:
# Import package
import os
from data import CreateDataLoader
from cycle_gan_model import CycleGANModel
from util.visualizer import save_images
from util import html
import config

In [ ]:
# Identify the path of your data folder
config.dataroot = THE_FULL_PATH_TO_TEMP_DATA # e.g '/home/yuzhe/temp_data'

In [ ]:
# Configuration
config.ntest = 2000
config.results_dir = './test_results/'
config.aspect_ratio = 1.0
config.eval = True
config.num_test = 50
config.phase = 'test'
config.model = 'test'
config.loadSize = config.fineSize
config.isTrain = False

config.num_threads = 1   
config.batch_size = 1    
config.serial_batches = True  
config.no_flip = True    
config.display_id = -1   # no visdom display

model = CycleGANModel()
model.initialize(config)
model.setup(config)
print("Network Model Created")
data_loader = CreateDataLoader(config)
dataset = data_loader.load_data()

In [ ]:
# Testing
# create a container for results
web_dir = os.path.join(config.results_dir, config.name, '%s_%s' % (config.phase, config.epoch))
webpage = html.HTML(web_dir, 'Experiment = %s, Phase = %s, Epoch = %s' % (config.name, config.phase, config.epoch))
    
for i, data in enumerate(dataset):
    if i >= config.num_test:
        break
    model.set_input(data)
    model.test()
    visuals = model.get_current_visuals()
    img_path = model.get_image_paths()
    if i % 5 == 0:
        print('processing (%04d)-th image... %s' % (i, img_path))
    save_images(webpage, visuals, img_path, aspect_ratio=config.aspect_ratio, width=config.display_winsize)

# Save results
webpage.save()